# 🔎 Moteur de Recherche – Corpus Discours US
### TD8 – Widgets Interactifs & TF-IDF  
**Projet réalisé par : Cyrine Nighaoui 


In [2]:
import pandas as pd
import sys
sys.path.append("../src")

from Corpus import Corpus

# Charger les données
df = pd.read_csv("discours_US.csv", sep="\t")

# Construire le corpus
corpus = Corpus("discours_US")

for i, row in df.iterrows():
    corpus.add_document(
        titre=row["descr"],
        auteur=row["speaker"],
        date=row["date"],
        url=row["link"],
        texte=row["text"]
    )

print("Corpus chargé :", corpus.ndoc, "documents et", corpus.naut, "auteurs")


Corpus chargé : 164 documents et 2 auteurs


In [3]:
import pickle

with open("engine.pkl", "rb") as f:
    engine = pickle.load(f)

print("Moteur TF-IDF chargé avec succès.")


Moteur TF-IDF chargé avec succès.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

query_input = widgets.Text(description="Query:", value="america")
k_slider = widgets.IntSlider(min=1, max=20, value=5, description="Top k:")

authors = sorted(df["speaker"].unique())
author_filter = widgets.Dropdown(options=["Tous"] + authors, description="Auteur:")

dates = sorted(df["date"].unique())
date_filter = widgets.Dropdown(options=["Toutes"] + dates, description="Date:")

btn = widgets.Button(description="Rechercher")

output = widgets.Output()


In [5]:
def clique_bouton(b):
    with output:
        clear_output(wait=True)

        query = query_input.value
        k = k_slider.value
        
        # Résultats bruts
        results = engine.search(query, top=9999)

        # Filtre par auteur
        if author_filter.value != "Tous":
            results = results[results["id"].apply(
                lambda doc_id: corpus.id2doc[doc_id].auteur == author_filter.value
            )]

        # Filtre par date
        if date_filter.value != "Toutes":
            results = results[results["id"].apply(
                lambda doc_id: corpus.id2doc[doc_id].date == date_filter.value
            )]

        # Limiter aux k premiers résultats
        results = results.head(k)

        display(results)


In [ ]:
display(query_input, k_slider, author_filter, date_filter, btn, output)
btn.on_click(clique_bouton)


Text(value='america', description='Query:')

IntSlider(value=5, description='Top k:', max=20, min=1)

Dropdown(description='Auteur:', options=('Tous', 'CLINTON', 'TRUMP'), value='Tous')

Dropdown(description='Date:', options=('Toutes', 'April 12, 2015', 'April 14, 2015', 'April 20, 2015', 'April …

Button(description='Rechercher', style=ButtonStyle())

Output()

## ✔ Interface prête  
Le moteur TF-IDF est pré-calculé et chargé instantanément via `engine.pkl`.  
L’utilisateur peut maintenant effectuer :
- une requête textuelle
- un choix de *top k*
- un filtre par auteur
- un filtre par date  
via une interface interactive basée sur `ipywidgets`.
